<a href="https://colab.research.google.com/github/Kei-Sanada/Numerai/blob/master/248_Modeling_20210125_2100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Round248にはこちらを提出**

In [1]:
# install dependencies
!pip install pandas sklearn numerapi

In [2]:
!free -h 

              total        used        free      shared  buff/cache   available
Mem:            12G        558M         11G        1.1M        473M         11G
Swap:            0B          0B          0B


In [3]:
import pandas as pd
import numerapi

In [4]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")

In [5]:
#training_data.head()

In [6]:
tournament_data  = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
#tournament_data.head()

In [7]:
#tournament_data['target_kazutsugi'].unique()

In [8]:
# import dependencies
import sklearn.linear_model

In [9]:
#concentrate in charisma, wisdom 
feature_names_wisdom = [f for f in training_data.columns if "feature_wisdom" in f]
feature_names_charisma = [f for f in training_data.columns if "feature_charisma" in f]
feature_names_intelligence = [f for f in training_data.columns if "intelligence33" in f]

In [10]:
# find only the feature columns
feature_cols = feature_names_wisdom + feature_names_charisma + feature_names_intelligence
# select those columns out of the training dataset
training_features = training_data[feature_cols]

In [11]:
# create a model and fit the training data (~30 sec to run)
model = sklearn.linear_model.LinearRegression()
model.fit(training_features, training_data.target)

LinearRegression()

In [12]:
# select the feature columns from the tournament data
live_features = tournament_data[feature_cols]

In [13]:
# predict the target on the live features
predictions_sklearn_linear_model = model.predict(live_features)

正規化 https://forum.numer.ai/t/model-diagnostics-feature-exposure/899

In [14]:
 #training_data["target_kazutsugi"]

In [15]:
"""
from xgboost import XGBRegressor
# train a model to make predictions on tournament data
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, 
       #colsample_bytree=1,#2021/01/24 Mod by Kei
       colsample_bytree=0.1, 
       gamma=0,
       importance_type='gain', 
       #learning_rate=0.1, #2020/11/29 Mod by Kei
       learning_rate=0.01,
       max_delta_step=0,
       #max_depth=3,
       #max_depth=2, #2020/11/15 Mod by Kei
       #max_depth=1, #2020/11/29 Mod by Kei
       max_depth=5, #2021/01/24 Mod by Kei
       #min_child_weight=1,
       #min_child_weight=3, #2020/11/15 Mod by Kei
       #min_child_weight=4, #2020/11/22 Mod by Kei
       min_child_weight=5, #2020/11/29 Mod by Kei
       missing=None, 
       #n_estimators=100, #2021/01/24 Mod by Kei
       n_estimators=2000,
       #n_jobs=1,#2021/01/24 Mod by Kei
       n_jobs=-1, 
       nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity =1,
       alpha=0.1) #2020/11/01 alpha=0.1　重みに関するL1正則化項高次元の場合に用いるらしい。
#model.fit(training_data[feature_names_dexterity], training_data["target_kazutsugi"])
#model.fit(training_data[feature_names_dexterity], training_data["target"]) mod by kei 2020/12/07
#model.fit(training_data[feature_names_wisdom + feature_names_intelligence], training_data["target"]) #mod by kei 2021/01/04
model.fit(training_data[feature_cols], training_data["target"])#2021/01/24 Mod by Kei
# submit predictions to numer.ai
predictions_XGBR = model.predict(tournament_data[feature_cols])
"""

'\nfrom xgboost import XGBRegressor\n# train a model to make predictions on tournament data\nmodel = XGBRegressor(base_score=0.5, booster=\'gbtree\', colsample_bylevel=1,\n       colsample_bynode=1, \n       #colsample_bytree=1,#2021/01/24 Mod by Kei\n       colsample_bytree=0.1, \n       gamma=0,\n       importance_type=\'gain\', \n       #learning_rate=0.1, #2020/11/29 Mod by Kei\n       learning_rate=0.01,\n       max_delta_step=0,\n       #max_depth=3,\n       #max_depth=2, #2020/11/15 Mod by Kei\n       #max_depth=1, #2020/11/29 Mod by Kei\n       max_depth=5, #2021/01/24 Mod by Kei\n       #min_child_weight=1,\n       #min_child_weight=3, #2020/11/15 Mod by Kei\n       #min_child_weight=4, #2020/11/22 Mod by Kei\n       min_child_weight=5, #2020/11/29 Mod by Kei\n       missing=None, \n       #n_estimators=100, #2021/01/24 Mod by Kei\n       n_estimators=2000,\n       #n_jobs=1,#2021/01/24 Mod by Kei\n       n_jobs=-1, \n       nthread=None, objective=\'reg:squarederror\', random

In [16]:
import lightgbm as lgb

# create a model and fit（LGBのハイパラは↑の公式XGBに寄せてみました）
params = {
            'n_estimators': 2000,
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'max_depth': 5,
            'learning_rate': 0.01, 
            'feature_fraction': 0.1,
            'seed': 42,
            'alpha':0.1
            }    
model = lgb.LGBMRegressor(**params)
model.fit(training_data[feature_names_wisdom + feature_names_intelligence], training_data["target"])
predictions_LGB_wisdom = model.predict(tournament_data[feature_names_wisdom + feature_names_intelligence])


[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1


In [17]:
# create a model and fit（LGBのハイパラは↑の公式XGBに寄せてみました）
params = {
            'n_estimators': 2000,
            'objective': 'regression',
            'boosting_type': 'gbdt',
            'max_depth': 5,
            'learning_rate': 0.01, 
            'feature_fraction': 0.1,
            'seed': 42,
            'alpha':0.1
            }    
model = lgb.LGBMRegressor(**params)
model.fit(training_data[feature_names_charisma + feature_names_intelligence], training_data["target"])
predictions_LGB_charisma = model.predict(tournament_data[feature_names_charisma + feature_names_intelligence])


[LightGBM] [Warning] feature_fraction is set=0.1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1


In [18]:
predictions = (predictions_sklearn_linear_model + predictions_LGB_wisdom + predictions_LGB_charisma) / 3

In [19]:
#tournament_data["id"]

In [20]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

,id,prediction_kazutsugi
0,n0003aa52cab36c2,0.499017
1,n000920ed083903f,0.499526
2,n0038e640522c4a6,0.505004
3,n004ac94a87dc54b,0.494756
4,n0052fe97ea0c05f,0.504223


In [21]:
import numpy as np
ranked_prediction = training_data["target"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data["target"], ranked_prediction)[0, 1]

In [22]:
ranked_prediction

0         0.249239
1         0.049854
2         0.049856
3         0.049858
4         0.750779
            ...   
501803    0.750773
501804    0.950146
501805    0.249237
501806    0.750775
501807    0.750777
Name: target, Length: 501808, dtype: float64

In [23]:
correlation

0.9095228541212714

In [24]:
#training_data["target_kazutsugi"]

In [25]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "CYATEL5QQBU6APNFLCV7HEE7PV6SC7V6"
secret_key = "Y22BTSUGU4JEFGQB3RZNEESSULKA3HQJPAW3KI6BIXH2AMNMCTC44IFWTOQIO2UW"
model_id = "3c77ba09-cfa2-4b18-b789-918340c84c82"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [26]:
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

2021-01-25 13:31:46,800 INFO numerapi.base_api: uploading predictions...


sklearn_linear_modelと　LightGBM
concentrate in charisma, wisdom 

DIAGNOSTICS
Performance
Validation Sharpe
0.8669
Validation Mean
0.0208
Feature Neutral Mean
0.0094
Risk
Validation SD
0.0239
Feature Exposure
0.3186
Max Drawdown
-0.064
MMC
Corr + MMC Sharpe
0.7359
MMC Mean
0.0053
Corr With Example Preds
0.5276